In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **IF YOU LIKE THIS NOTEBOOK PLEASE DO UPVOTE!!!**
# *feel free to suggest what else i can do to make it better*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,mean_squared_error
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot

from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


In [ ]:
df=pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
df.head()

**FIRST WE WILL CHECK IS THERE ANY NULL VALUES**

In [ ]:
df.isnull().sum()

**Through visualization we can see there are no null values in our dataset**

In [ ]:
sns.heatmap(df.isnull())

> There are no categorical values we have only numeric values

In [ ]:
df.info()

In [ ]:
df.describe()

**Let's check how many columns has only 2 unique values**

In [ ]:
for i in df.columns:
    if df[i].nunique() < 3:
        print(i)

**anaemia**

**diabetes**

**high_blood_pressure**

**sex**

**smoking**

**DEATH_EVENT**
**are the columns who have 1 and 0 value in it**

> **Let's do EDA**

**As we can see the maximum number people are between 50-70 age criteria**

In [ ]:
fig=ff.create_distplot([df['age'].values],['age'])
fig.update_layout(title_text='Age Distribution plot')
fig.show()

> **Normal serum_sodium ranges between 135-143**

In [ ]:
fig=ff.create_distplot([df['serum_sodium'].values],['serum_sodium'])
fig.update_layout(title_text='serum_sodium Distribution plot')
fig.show()


> **we are comparing time with death event Blue represents person died and red represents person survived**

In [ ]:
fig = px.histogram(df, x="time", color="DEATH_EVENT", marginal="box", hover_data=df.columns)
fig.show()

> Maximum people died due to Heart failure at the age of 60 and 70

> Minimum people survied are at the age of 50 and 60

In [ ]:
surv = df[df["DEATH_EVENT"]==0]["age"]
not_surv = df[df["DEATH_EVENT"]==1]["age"]
hist_data = [surv,not_surv]
group_labels = ['Survived', 'Not Survived']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.5)
fig.update_layout(title_text="Analysis in Age on Survival Status")
fig.show()

**Death is mostly due to**

**Gender**

**Diabetes**

**Smoking**

> let's plot all of the these important features with Age column and correlate to Death event

> lets calculate how many people has diabetes has correlation to death event

In [ ]:
fig=px.violin(df,x='diabetes',y='age',color='DEATH_EVENT',points='all',box=True)
fig.update_layout(title_text='Analysis in Age and Diabetes on Survival Status')
fig.show()

In [ ]:
fig=px.violin(df,x='sex',y='age',color='DEATH_EVENT',points='all',box=True)
fig.update_layout(title_text='Analysis in Age and Gender on Survival Status')
fig.show()

> let's plot Age and somking with death event(we will understand wether person who used to smoke has more death occur

In [ ]:
fig=px.violin(df,x='smoking',y='age',color='DEATH_EVENT',points='all',box=True)
fig.update_layout(title_text='Analysis in Age and smoking on Survival Status')
fig.show()

**now let's plot other columns wrt to death event**

In [ ]:
fig=px.histogram(df,x='creatinine_phosphokinase',color='DEATH_EVENT',marginal='box')
fig.update_layout(title_text='Analysis of creatinine_phosphokinase wrt Death Event')
fig.show()

In [ ]:
fig=px.histogram(df,x='ejection_fraction',color='DEATH_EVENT',marginal='box')
fig.update_layout(title_text='Analysis of ejection_fraction wrt Death Event')
fig.show()

In [ ]:
fig=px.histogram(df,x='platelets',color='DEATH_EVENT',marginal='box')
fig.update_layout(title_text='Analysis of platelets wrt Death Event')
fig.show()

In [ ]:
fig = px.histogram(df, x="serum_creatinine", color="DEATH_EVENT", marginal="box", hover_data=df.columns)
fig.show()

In [ ]:
fig = px.histogram(df, x="serum_sodium", color="DEATH_EVENT", marginal="box",hover_data=df.columns)
fig.show()

**Detailed plots for calculating survived and death counts**

> Male survival percent - 44.1%

> Male death percent - 20.7%

> Female survival percent - 23.7%

> Female death percent - 11.4%

In [ ]:
male = df[df["sex"]==1]
female = df[df["sex"]==0]


labels = ['Male - Survived','Male - Not Survived', "Female -  Survived", "Female - Not Survived"]
values = [len(male[df["DEATH_EVENT"]==0]),len(male[df["DEATH_EVENT"]==1]),
          len(female[df["DEATH_EVENT"]==0]),len(female[df["DEATH_EVENT"]==1])]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(title_text="Analysis on Survival - Gender")
fig.show()

In [ ]:
diabetes = df[df["diabetes"]==1]["age"]
not_diabetes = df[df["diabetes"]==0]["age"]
hist_data = [diabetes,not_diabetes]
group_labels = ['Diabetes', 'Not having Diabetes']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.5)
fig.update_layout(title_text="Analysis in Age on Diabetes Status")
fig.show()

> 58.2% people don't have diabetes 

> 41.8% people have diabetes 

In [ ]:
labels = ['No Diabetes','Diabetes']
diabetes_yes = df[df['diabetes']==1]
diabetes_no = df[df['diabetes']==0]
values = [len(diabetes_no), len(diabetes_yes)]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(
    title_text="Analysis on Diabetes")
fig.show()

In [ ]:
yes_diabetes=df[df['diabetes']==1]
no_diabetes=df[df['diabetes']==0]

dia_deaths=yes_diabetes[df['DEATH_EVENT']==1]
dia_live=yes_diabetes[df['DEATH_EVENT']==0]
no_dia_deaths=no_diabetes[df['DEATH_EVENT']==1]
no_dia_live=no_diabetes[df['DEATH_EVENT']==0]
labels=['died having diabetes','survived having diabetes','died not having diabetes','survived not having diabetes']
values=[len(dia_deaths),len(dia_live),len(no_dia_deaths),len(no_dia_live)]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(title_text='testing')
fig.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(df.corr(),annot=True)

> let's separate the dependent and independent columns

In [ ]:
X=X=df.drop('DEATH_EVENT',axis=1)
y=df['DEATH_EVENT']
print(X.shape)
print(y.shape)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# **Models**

In [ ]:
print(len(X_train))
print(len(y_train))

> **Logistic Regression**

In [ ]:
model=LogisticRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print("Classification Report is:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("testing Score:\n",model.score(X_test,y_test)*100)
print("Mean Squared Error:\n",mean_squared_error(y_test,y_pred))
print(model.get_params())

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
params={
    'penalty':['l1','l2','elasticnet,none'],
    'C':[0.1,1,5,10,100],
    'solver':['neton-cg','lbfgs','liblinear','sag','saga'],
    'max_iter':[50,100,1000,2000,5000]
}
model1=GridSearchCV(model,param_grid=params,cv=5,n_jobs=-1)
model1.fit(X_train,y_train)
print(model1.best_estimator_)
print(model1.best_params_)
y_pred=model1.predict(X_test)
print("Classification Report is:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("Training Score:\n",model.score(X_train,y_train)*100)
print("Mean Squared Error:\n",mean_squared_error(y_test,y_pred))


> **SVC**

In [ ]:
model=SVC()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print("Classification Report is:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("testing Score:\n",model.score(X_test,y_test)*100)
print("Mean Squared Error:\n",mean_squared_error(y_test,y_pred))
print(model.get_params())

> **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)
y_pred=dtree.predict(X_test)
print("Classification Report is:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("testing Score:\n",dtree.score(X_test,y_test)*100)
print("Mean Squared Error:\n",mean_squared_error(y_test,y_pred))


> **Random Forest Classifier**

In [ ]:
model=RandomForestClassifier()
model.fit(X_train,y_train)
y_pred=dtree.predict(X_test)
print("Classification Report is:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("Training Score:\n",model.score(X_train,y_train)*100)
print("Mean Squared Error:\n",mean_squared_error(y_test,y_pred))
print(model.get_params())
print('accuracy score',accuracy_score(y_test,y_pred)*100)

> **Gradient boosting**

In [ ]:
model=GradientBoostingClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print("Classification Report is:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("Training Score:\n",model.score(X_train,y_train)*100)
print("Mean Squared Error:\n",mean_squared_error(y_test,y_pred))
print(model.get_params())
print('accuracy score',accuracy_score(y_test,y_pred)*100)

> **XGB classifier**

In [ ]:
model1=XGBClassifier()
model1.fit(X_train,y_train)
y_pred=model1.predict(X_test)
print("Classification Report is:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("Training Score:\n",model1.score(X_train,y_train)*100)
print("Mean Squared Error:\n",mean_squared_error(y_test,y_pred))
print(model1.get_params())
print('accuracy score',accuracy_score(y_test,y_pred)*100)

> **Extra tree classifier**

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
abc = ExtraTreesClassifier()
abc.fit(X_train,y_train)
y_pred=model1.predict(X_test)
print("Classification Report is:\n",classification_report(y_test,y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("Training Score:\n",abc.score(X_train,y_train)*100)
print("Mean Squared Error:\n",mean_squared_error(y_test,y_pred))
print(abc.get_params())
print('accuracy score',accuracy_score(y_test,y_pred)*100)

**Logistic Regression -- 80%**

**SVC -- 58%**

**Decision Tree -- 63%**

**Random Forest Classifier -- 63%**

**Gradient Boosting -- 75%**

**XGBoost -- 73%**

**Extra tree Classifier -- 73%**

# **If you like the notebook please do UPVOTE!!!!!**